In [1]:
import math
import numpy as np
import sys
import math
import time

In [2]:
def RemplirPremiereLigne():
    liste = []
    for number in range(1,tcc):
        for j in range(1,tcc):
            for i in range(1,tcc):
            
                liste.append('x' + str(i)+ str(j)+ str(number))
                liste.append('not x' + str(i)+ str(j)+ str(number))
    return liste

In [3]:
def neutre(clauses):
    X = an_array.copy()
    for i in clauses:
        retenu = 0
        if i.startswith("not "):
            retenu = 1
            i = i[4:]
        pos = (int(i[3])-1)* (2 * tc*tc*tc*tc )  + (int(i[1])-1)*2 + (int(i[2])-1)* (2 * tc*tc) + retenu
        X[pos] = 0
    return X

##################################################################################################################

def splitter(l, arr, clauses):
    d = l.copy()
    for elem in l[0:len(l)]:
        del d[0]
        if d == []:
            break
        if (elem+d[0]) in tabdouble:
            continue
        else:
            tabdouble.append((elem+d[0]))
        for e in d:
            temp = ["not " + elem, "not " + e]
            arrtemp = neutre(temp)
            arr = np.vstack([arr, arrtemp])
    return arr

In [4]:
tabdouble = []
def SudokuClauses():
    listos = an_array.copy()
    
    for i in range(1,tcc):
         for j in range(1,tcc):
            tab =[]
            for z in range(1,tcc):
                tab.append('x'+str(i)+str(j)+str(z))
            mm = neutre(tab)
            listos = np.vstack([listos, mm])
            a = splitter(tab, an_array, listos)
            if len(a) != L:
                a = np.delete(a, 0, 0)
                listos = np.vstack([listos, a])
        
        
    for i in range(1,tcc):
        for j in range(1,tcc):
            for z in range(1,tcc):
                clausehori = []
                clauseverti = []
                for x in range(1,tcc):
                    clausehori.append('x'+str(x)+str(j)+str(z))
                for y in range(1,tcc):
                    clauseverti.append('x'+str(i)+str(y)+str(z))

                
                neutreverti = neutre(clauseverti)
                
                listos = np.vstack([listos, neutreverti])
                    
                a = splitter(clauseverti, an_array, listos)
                if len(a) != L:
                    a = np.delete(a, 0, 0)
                    listos = np.vstack([listos, a])
                    
                neutrehori = neutre(clausehori)
                
                listos = np.vstack([listos, neutrehori])
                
                b = splitter(clausehori, an_array, listos)
                if len(b) != L:
                    b = np.delete(b, 0, 0)
                    listos = np.vstack([listos, b])

    for z in range (1,tcc):
        for iplus in range(0,tc*tc,tc):
            for jplus in range(0,tc*tc,tc):
                c = []
                for i in range(1,tc+1):
                    for j in range(1,tc+1):
                        c.append('x'+str(i + iplus)+str(j + jplus)+str(z))

                nc = neutre(c)
                listos = np.vstack([listos, nc])  
                a = splitter(c, an_array, listos)
                if len(a) != L:
                    a = np.delete(a, 0, 0)
                    listos = np.vstack([listos, a])

    listos =  np.delete(listos, 0, 0)
    return listos

In [5]:
def spegrille(gri, cl):
    listos2 = an_array.copy()
    for m in range(tc*tc):
        for j in range(tc*tc):
            if gri[m][j] != 0:
                X = neutre(['x'+str(j+1)+str(m+1)+str(grille[m][j])])
                listos2 = np.vstack([listos2, X])
    listos2 =  np.delete(listos2, 0, 0)
    return np.vstack([listos2, cl])

In [6]:
def choisirvariable(clauses):
    for i in clauses:
        if np.sum(i==0) == 1:
            if np.sum(i==1) == 0:
                getindex = np.where(i == 0)
                #print(index[getindex[0][0]])
                return getindex[0][0]
    for i in range(len(index)):
        if 0 in clauses.T[i]:
            return i
        
def choisirvaleur(clauses, var):
    return 1

def Affecter(val, var, clauses):
    clauses.T[var] = np.where( (clauses.T[var] < 2) , val , 2 )
    if (index[var]).startswith('not '):
        aj = -1
    else:
        aj = 1
    clauses.T[var+aj] = np.where( (clauses.T[var+aj] < 2) , val*-1 , 2 )

In [7]:
def resolveur(clauses):
    stop = 0
    while stop !=1:
        c = 0
        for i in clauses:
            if np.sum(i==0) == 1:
                if np.sum(i==1) == 0:
                    c = 1
                    getindex = np.where(i == 0)
                    idx = getindex[0][0]
                    clauses.T[idx] = np.where( (clauses.T[idx] < 2) , 1 , 2 )
                    if (index[idx]).startswith('not '):
                        aj = -1
                    else:
                        aj = 1
                    clauses.T[idx+aj] = np.where( (clauses.T[idx+aj] < 2) , -1 , 2 )
        if c == 0:
            stop = 1
    return 0

In [8]:
def DPLL(clauses, r):
    # si toutes les lignes contiennent au moins un 1, alors retourner oui
    if r == 0:
        resolveur(clauses)
        
    skip = 0
    for line in clauses:
        if 1 not in line:
            skip = 1
            if 0 not in line:
                return 0
                
    if skip == 0:
        return 1
    
    #print(vvv)
    #si une ligne contient uniquement des -1, alors retourner non
    X = choisirvariable(clauses)
    V = choisirvaleur(clauses, X)
    Affecter(V,X,clauses)
    if DPLL(clauses,1) == 1:
        return 1
    Affecter(V*-1,X,clauses)
    
    if DPLL(clauses,1) == 1:
        return 1
    Affecter(0,X,clauses)
    return 0

In [9]:
diabolique =[[0,1,0, 7,0,0, 2,0,8],
            [9,0,0, 0,5,0, 0,0,0],
            [0,0,0, 0,0,0, 0,3,0],
            [0,0,2, 0,0,7, 0,0,0],
            [0,8,0, 2,0,0, 1,0,6],
            [0,0,0, 0,0,0, 4,0,0],
            [1,0,0, 0,0,9, 6,0,3],
            [0,3,0, 0,0,0, 0,7,0],
            [0,0,0, 6,0,0, 0,4,0]]

facile = [[0,6,0,4,0,0,9,1,0],
         [0,4,2,0,0,1,6,8,7],
         [1,0,0,6,0,8,0,5,0],
         [0,7,0,2,6,0,8,9,0],
         [0,0,0,3,0,4,0,0,6],
         [9,0,0,8,1,0,0,0,5],
         [2,1,9,7,0,0,0,0,8],
         [7,0,0,5,8,0,0,0,0],
         [0,8,0,0,2,3,0,0,9]]

moyen = [[0,0,0,0,0,0,0,0,1],
         [7,0,0,0,9,0,8,5,2],
         [0,0,3,5,0,1,7,0,0],
         [0,3,0,0,6,0,5,0,0],
         [6,0,5,2,0,3,1,0,4],
         [0,0,8,0,1,0,0,7,0],
         [0,0,1,9,0,8,4,0,0],
         [3,8,7,0,4,0,0,0,5],
         [4,0,0,0,0,0,0,0,0]]

difficile = [[0,0,6,0,0,0,3,7,0],
             [0,0,5,7,0,0,0,0,9],
             [0,0,0,1,0,6,0,0,2],
             [0,0,0,4,0,1,0,5,0],
             [0,3,0,2,0,8,0,1,0],
             [0,8,0,5,0,7,0,0,0],
             [4,0,0,8,0,9,0,0,0],
             [7,0,0,0,0,5,9,0,0],
             [0,9,2,0,0,0,7,0,0]]

démoniaque = [[0,3,9,0,0,0,0,0,2],
              [1,0,0,0,8,9,3,0,0],
              [5,0,0,0,0,0,4,8,0],
              [0,0,0,0,5,1,0,3,0],
              [0,7,0,3,0,2,0,4,0],
              [0,5,0,8,4,0,0,0,0],
              [0,1,5,0,0,0,0,0,4],
              [0,0,2,4,7,0,0,0,5],
              [4,0,0,0,0,0,2,7,0]]

grille = démoniaque
tc = int(math.sqrt(len(grille[0])))
tcc = int(tc * tc +1)
index = RemplirPremiereLigne()
#listosX = np.insert(a, 0, 'nan', axis = 0)
an_array = np.zeros((len(index)))
an_array[:] = 2
L = len(an_array)

In [10]:
tabdouble = []
start_time = time.time()
Clauses = SudokuClauses()
print("%s seconds" % (time.time() - start_time))
start_time = time.time()
CC = spegrille(grille, Clauses)
DPLL(CC,0)
print("%s seconds" % (time.time() - start_time))

43.884615898132324 seconds
17.966617107391357 seconds


In [11]:
def remplirGrille(clauses, gri):
    newgrille = gri.copy()

    for i in range(len(index)):
        if 1 in clauses.T[i]:
            s = index[i]
            if s.startswith("not "):
                pass
            else:
                newgrille[int(s[2])-1][int(s[1])-1] = int(s[3])
    
    return newgrille
rep = remplirGrille(CC, grille)
for i in rep:
    print(i)

[8, 3, 9, 5, 6, 4, 7, 1, 2]
[1, 2, 4, 7, 8, 9, 3, 5, 6]
[5, 6, 7, 1, 2, 3, 4, 8, 9]
[2, 4, 8, 6, 5, 1, 9, 3, 7]
[6, 7, 1, 3, 9, 2, 5, 4, 8]
[9, 5, 3, 8, 4, 7, 6, 2, 1]
[7, 1, 5, 2, 3, 6, 8, 9, 4]
[3, 9, 2, 4, 7, 8, 1, 6, 5]
[4, 8, 6, 9, 1, 5, 2, 7, 3]
